I worked with this notbeook inside the development data folder, so if you would like to run the code, download it and put it in the right folder (or change the paths in the notebook).

In [1]:
import pprint
import json
import codecs
from nltk.tree import Tree
from collections import defaultdict

In [2]:
# Anayze development data:

with codecs.open ('relations.json', 'r', encoding = 'utf-8') as pdtb_file:
    

    # Assign all relations (a list) to a variable
    relations = [json.loads(x) for x in pdtb_file];

In [3]:
# Loop through relations and select only explicit relations:

# List for explicit relations:

relations_explicit = []

for relation in relations:
    if relation['Type'] == 'Explicit':
        relations_explicit.append(relation)
        
print(len(relations_explicit))
pprint.pprint(relations_explicit[0])

680
{'Arg1': {'CharacterSpanList': [[517, 564]],
          'RawText': 'to restrict the RTC to Treasury borrowings only',
          'TokenList': [[517, 519, 85, 2, 3],
                        [520, 528, 86, 2, 4],
                        [529, 532, 87, 2, 5],
                        [533, 536, 88, 2, 6],
                        [537, 539, 89, 2, 7],
                        [540, 548, 90, 2, 8],
                        [549, 559, 91, 2, 9],
                        [560, 564, 92, 2, 10]]},
 'Arg2': {'CharacterSpanList': [[573, 629]],
          'RawText': 'the agency receives specific congressional authorization',
          'TokenList': [[573, 576, 95, 2, 13],
                        [577, 583, 96, 2, 14],
                        [584, 592, 97, 2, 15],
                        [593, 601, 98, 2, 16],
                        [602, 615, 99, 2, 17],
                        [616, 629, 100, 2, 18]]},
 'Connective': {'CharacterSpanList': [[566, 572]],
                'RawText': 'unless',
         

In [4]:
# Split in two lists according to arg1 in same sentence and arg1 in previous sentence:

relations_ss = []
relations_ps = []
relations_other = []

for relation in relations_explicit:
    sentence_id_arg1 = relation['Arg1']['TokenList'][0][3]
    sentence_id_connective = relation['Connective']['TokenList'][0][3]
    sentence_id_arg2 = relation['Arg2']['TokenList'][0][3]
    
    if sentence_id_arg1 == sentence_id_connective == sentence_id_arg2:
        relations_ss.append(relation)
    elif int(sentence_id_arg1) == int(sentence_id_connective) - 1 == int(sentence_id_arg2) -1:
        relations_ps.append(relation)
    else: 
        relations_other.append(relation)


In [5]:
# Open parse file:

with codecs.open('parses.json', 'r', encoding = 'utf8') as parse_file:
    parses = json.load(parse_file)

In [6]:
def normalize_string(text):
    """"""
    
    clean_text_list = []
    punctuation = [',', '.', '!', '?', '"', ':',  '`' , ')', '(', '$', '#', '^', '*']
    apostrophe = "'"
    
  
    for punct in punctuation:
        if punct in text:
            text = text.replace(punct, '')
        if apostrophe in text:
            text = text.replace(apostrophe, ' ')
    text_no_punct = text
    
    text_list = text_no_punct.split()
    
    clean_text_list = []
    
    for word in text_list:
        word = word.strip()
        if word == 'didn':
            word = 'did n'
        elif word == 'doesn':
            word = 'does n'
        elif word == 'don':
            word = 'do n'
        clean_text_list.append(word)
    
    clean_text = ' '.join(clean_text_list)
    
    return(clean_text)

my_rel = relations_ss[4]

arg1_str = my_rel['Arg1']['RawText']
print(arg1_str)
print(normalize_string(arg1_str))
print(normalize_string("rtc's! work didn t doesn t"))

Absent other working capital the RTC would be forced to delay other thrift resolutions
Absent other working capital the RTC would be forced to delay other thrift resolutions
rtc s work did n t does n t


In [7]:
def get_const_type(constituent_list, constituent_name):
    """
    Input: set of constituents, name of a constituent type (str)
    Output: list of constituents of the selected type 
    """
    const_type_list = []
    
    for constituent in constituent_set:
        
        
        if constituent.startswith('('+constituent_name):
            const_type_list.append(constituent)
    
    return const_type_list
            
# E.g. get clauses:   

In [8]:
def get_subtrees(parsed_sentence, const_type = 0):
    """"""
    tree_nltk = Tree.fromstring(parsed_sentence)
    
    tree_height = tree_nltk.height()
    
    

    
        
    const_tuple_list = []

    for s in tree_nltk.subtrees():

        label = s.label()
        subtree_tokens_str = str(s.flatten())
        clean_subtree_str = normalize_string(subtree_tokens_str).lstrip(label).lower()


        if const_type != 0:
            if label == const_type:
                const_tuple_list.append((clean_subtree_str, label.lower()))

        else:

            const_tuple_list.append((clean_subtree_str, label.lower()))

    

    return const_tuple_list

doc_id = my_rel['DocID']
sentence_id_arg1 = my_rel['Arg1']['TokenList'][0][3]
parsed_sentence = parses[doc_id]['sentences'][sentence_id_arg1]['parsetree']  

consts = get_subtrees(parsed_sentence)



In [54]:
def get_arg_list(relation, arg = 'Arg1'):
    
    """"""
    
   
    # Dict with argument as key and constituent as value ( [label, constituent])
    # constituent_dict = defaultdict(list)

    doc_id = relation['DocID']
    sentence_id_arg1 = relation['Arg1']['TokenList'][0][3]
    #sentence_id_arg2 = relation['Arg2']['TokenList'][0][3]


    arg1_str = relation['Arg1']['RawText']
    #print(arg1_str)

     # Get sentence token id for argument tokens from relations file:

    sentence_token_id_relations = []
    relations_token_list = relation['Arg1']['TokenList']
    for line in relations_token_list:
        sentence_token_id = line[4]
        sentence_token_id_relations.append(sentence_token_id)

    # Give word in parsing a sentence id


    parsed_sentence_list = []


    #punct_list = [',', '``', '.', "'", '!', '`']

    clean_word_list = []
    for word in parses[doc_id]['sentences'][sentence_id_arg1]['words']:
        clean_word_list.append(word[0])

    # Map argument token to sentence_token_id
    argument_dict = dict()

    for sentence_token_id, word in enumerate(clean_word_list):
        if sentence_token_id in sentence_token_id_relations:

            argument_dict[sentence_token_id] = word


    argument_tuples = list(argument_dict.items())

    sorted_argument_tuples = sorted(argument_tuples)
    argument_list = []
    for n, a in sorted_argument_tuples:

        argument_list.append(a)

    # Check if the argument is interrupted (for instance by an attribution relation)

    for index, pair in enumerate(sorted_argument_tuples):
        if pair == sorted_argument_tuples[-1]:
            break

        next_pair = sorted_argument_tuples[index + 1]


        if (next_pair[0] - pair[0]) > 1:
            slice_index = index
            
            break
        else:
            slice_index = None
            
    # Get the argument as either 1 string in a list (if it is not interrupted) or as 2 strings in a list
    # if it is interrupted

    if slice_index:
        arg1_1 = ' '.join(argument_list[:slice_index + 1])

        arg1_2 = ' '.join(argument_list[slice_index + 1:])

        arg1 = [arg1_1, arg1_2]
    else:

        arg1 = [' '.join(argument_list)]
    
    
    return arg1

for relation in relations_ss:
    print(get_arg_list(relation))

['to restrict the RTC to Treasury borrowings only']
['to maintain the bad assets of thrifts that are sold']
['That debt would be paid off']
['The agency has already spent roughly $ 19 billion selling 34 insolvent S&Ls']
['Absent other working capital', 'the RTC would be forced to delay other thrift resolutions']
['We would have to wait']
['We would have to wait until we have collected on those assets']
["the RTC 's total obligations may not exceed $ 50 billion"]
['but that figure is derived']
["But Congress did n't anticipate or intend more public debt"]
['The RTC is going to have to pay a price of prior consultation on the Hill']
['but decided not to']
['the investment firm ordered full pages in the Monday editions of half a dozen newspapers']
['which already was operating']
['to stay in the market']
['In the aftermath of the 1987 debacle , the brokerage firm began taping commercials in-house']
['to tape a commercial']
["the true value has n't changed"]
['With the market fluctuating']

In [65]:
# Map arguments to constituents

def constituent_structure(relation, arg):
    """"""

    #myrelation = relations_ss[6]

    # Dicts with argument as key and constituent as value (tuple: (label, constituent))
    arg_is_const = defaultdict(list)
    arg_starts_const = defaultdict(list)
    arg_in_const = defaultdict(list)
    const_part_arg = defaultdict(list)

    


    doc_id = relation['DocID']
    sentence_id_arg = relation[arg]['TokenList'][0][3]
    #sentence_id_arg2 = relation['Arg2']['TokenList'][0][3]


    arg_str = relation[arg]['RawText']


    sentence = parses[doc_id]['sentences'][sentence_id_arg]['parsetree']

    const_list = get_subtrees(sentence)


    #const_starts_list = []
    #const_in_list = []

    arg_list = get_arg_list(relation)


    for argument in arg_list:
        #print(argument)
        argument = normalize_string(argument).lower().strip()
        print('the argument:', argument)



        for constituent in const_list:

            const, label = constituent
            const = const.strip()
            #print(const)


            # Constituents the argument is part of:

            if const == argument:

                arg_is_const[argument].append((label, const))




            elif const.startswith(argument):

                arg_starts_const[argument].append((label, const))


            elif argument in const:

                arg_in_const[argument].append((label, const))



            # Constituents that are part of the argument

            if const in argument:
                const_part_arg[argument].append((label, const))
                
    return arg_is_const, arg_starts_const, arg_in_const, const_part_arg



In [66]:
constituent_structure(relations_ss[4], 'Arg1')

the argument: absent other working capital
the argument: the rtc would be forced to delay other thrift resolutions


(defaultdict(list,
             {'absent other working capital': [('pp',
                'absent other working capital')]}),
 defaultdict(list,
             {'absent other working capital': [('',
                'absent other working capital he said the rtc would be forced to delay other thrift resolutions until cash could be raised by selling the bad assets'),
               ('s',
                'absent other working capital he said the rtc would be forced to delay other thrift resolutions until cash could be raised by selling the bad assets')]}),
 defaultdict(list,
             {'the rtc would be forced to delay other thrift resolutions': [('',
                'absent other working capital he said the rtc would be forced to delay other thrift resolutions until cash could be raised by selling the bad assets'),
               ('s',
                'absent other working capital he said the rtc would be forced to delay other thrift resolutions until cash could be raised by selling the b